# Contents

## Importing Libraries and Data (orders_products_busy.pkl)
## Aggregated mean of 'order_number' by 'department_id'
## Creating a loyalty flag
### &emsp; Creating a 'max_order' column
### &emsp; Creating the 'loyalty_flag' column
## Creating a spending flag
### &emsp; Creating an 'avg_price' column
### &emsp; Creating the 'spending_flag' column
## Creating an order frequency flag
### &emsp; Creating a 'median_days_since_prior_order' column
### &emsp; Creating the 'order_frequency_flag' column
## Exporting data as orders_products_agg.pkl
## Appendix: Consistency check on the 'price' column discovered during Exercise 4.9

# Step 1: Importing Libraries and Dataframes

In [1]:
# Importing Libraries

import pandas as pd
import numpy as np
import os

In [2]:
# Turning project folder into a string

path = r'C:\Users\davau\OneDrive - College of the Sequoias\Career Foundry\Data Immersion\Achievement 4 (Python)\Instacart Basket Analysis'

In [3]:
# Importing the main dataframe that I exported after Task 4.7

df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_busy.pkl'))

In [4]:
df_ords_prods_merge.shape

(32404859, 19)

In [5]:
df_ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,Most orders


# Step 2

In [6]:
# Finding the aggregated mean of 'order_number' by 'department_id'

df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


# Step 3

Looking through the output, I notice that department_id=21 (missing) is the one with the largest mean order number; however, as this is not actually a department but rather an indication that there was no department listed with the item, the most popular department is really department_id=10 (bulk).  Meanwhile, department_id=5 (alcohol) is the one with the smallest mean order number (the least popular department).  Moreover, the means are all fairly close, with all but 3 departments having a mean between 15 and 20.

These results differ from those of the subset in a couple of ways.  One, the results for the subset did not include all departments, whereas these results do.  Two, the reported means are different because the subset didn't contain all the data.


# Step 4: Creating a loyalty flag

## Creating a 'max_order' column to use for creating the loyalty flag later

In [7]:
# Creating the `max_order' column

df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [8]:
# Checking to make sure it was added

df_ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10


## Creating the 'loyalty_flag' column

In [9]:
# Customers with over 40 orders labeled 'Loyal customer'

df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [10]:
# Customers with between 11 and 40 orders (inclusive) labeled 'Regular customer'

df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [11]:
# Customers with 10 or fewer orders labeled 'New customer'

df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [12]:
# Checking the new 'loyalty_flag' column

df_ords_prods_merge.head(20)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,77,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,77,7,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
6,550135,1,prior,7,1,9,20.0,196,1,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,77,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer


In [13]:
df_ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

# Step 5

In [14]:
# Checking basic statistics on the three types of customers flagged above. 

df_ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['min', 'max', 'sum','mean']})

prices                                 
                    min      max          sum       mean
loyalty_flag                                            
Loyal customer      1.0  99999.0  106814042.2  10.386336
New customer        1.0  99999.0   83011787.2  13.294670
Regular customer    1.0  99999.0  198391693.2  12.495717

On average, loyal customers choose cheaper products than new and regular customers.  Regular customers contributed most to Instacart revenue, but this is likely due to the fact that there are more of them than loyal customers.

# Step 6

## Creating an 'avg_price' column to use for creating the 'spending' flag later

In [15]:
# Creating the 'avg_price' column, which displays the mean price of the items purchased by each user

df_ords_prods_merge['avg_price'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [16]:
# Checking the new 'avg_price' column

df_ords_prods_merge.head(15)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797
2,473747,1,prior,3,3,12,21.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797
4,431534,1,prior,5,4,15,28.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
6,550135,1,prior,7,1,9,20.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer,6.367797
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,7,9.0,both,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer,6.367797
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,7,9.0,both,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer,6.367797
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797


## Creating the 'spending_flag' column

In [17]:
# If avg_price < 10, labeled 'Low spender'

df_ords_prods_merge.loc[df_ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'

In [18]:
# If avg_price >= 10, labeled 'High spender'

df_ords_prods_merge.loc[df_ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

In [19]:
# Checking the new 'spending_flag' column

df_ords_prods_merge.head(20)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797,Low spender
2,473747,1,prior,3,3,12,21.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low spender
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender
4,431534,1,prior,5,4,15,28.0,196,1,1,...,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low spender
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
6,550135,1,prior,7,1,9,20.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer,6.367797,Low spender
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,9.0,both,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer,6.367797,Low spender
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,9.0,both,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer,6.367797,Low spender
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender


In [20]:
# Getting a frequency distribution for 'spending_flag'

df_ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

# Step 7

## Creating a 'median_days_since_prior_order' column to use for creating the 'order frequency' flag later

In [21]:
# Creating the 'median_days_since_prior_order' column, 
# which displays the median number of days each user takes between orders

df_ords_prods_merge['median_days_since_prior_order'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [22]:
# Checking the new 'median_days_since_prior_order' column

df_ords_prods_merge.head(20)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_days_since_prior_order
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5
2,473747,1,prior,3,3,12,21.0,196,1,1,...,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5
4,431534,1,prior,5,4,15,28.0,196,1,1,...,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5
6,550135,1,prior,7,1,9,20.0,196,1,1,...,both,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer,6.367797,Low spender,20.5
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,both,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer,6.367797,Low spender,20.5
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,both,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer,6.367797,Low spender,20.5
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5


## Creating the 'order_frequency_flag' column

In [23]:
# If the median days since prior order is higher than 20, 
# labeled 'Non-frequent customer'

df_ords_prods_merge.loc[df_ords_prods_merge['median_days_since_prior_order'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [24]:
# If the median is between 11 and 20 (inclusive), 
# labeled 'Regular customer'

df_ords_prods_merge.loc[(df_ords_prods_merge['median_days_since_prior_order'] > 10) & (df_ords_prods_merge['median_days_since_prior_order'] <= 20), 'order_frequency_flag'] = 'Regular customer'

In [25]:
# If the median is <= 10, labeled 'Frequent customer'

df_ords_prods_merge.loc[df_ords_prods_merge['median_days_since_prior_order'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [26]:
# Checking the new 'order_frequency_flag' column

df_ords_prods_merge.head(20)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_days_since_prior_order,order_frequency_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
6,550135,1,prior,7,1,9,20.0,196,1,1,...,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,Mid-range product,Regularly busy,Busiest,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [27]:
# Getting a frequency distribution for 'order_frequency_flag'

df_ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: order_frequency_flag, dtype: int64

# Step 9

In [28]:
# Exporting df_ords_prods_merge as orders_products_agg.pkl

df_ords_prods_merge.to_pickle(os.path.join(path,'02 Data','Prepared Data','orders_products_agg.pkl'))

# New discoveries during 4.9

In 4.9, the high max price was noted while trying to make a histogram of the 'price' column.  It was discovered that there were two categories of items with unreasonably high prices: some cost 14,900 and others cost 99,999.  It was decided to treat those as missing values, so they were replaced with NaNs.  The orders_products_agg.pkl exported above does not reflect this change, as it wasn't made until 4.9.  However, the changes have now been exported under the new file name _______ .  The code below was used to replace these high prices with NaNs.

In [29]:
# Replacing prices over $100 with NaNs

ords_prods_agg.loc[ords_prods_agg['prices'] >100, 'prices'] = np.nan

NameError: name 'ords_prods_agg' is not defined